<a href="https://colab.research.google.com/github/vincelemke99/Bachelor-Thesis/blob/main/lead_predication_final_2_0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pandas category_encoders scikit-learn --q




     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.9/81.9 kB 1.6 MB/s eta 0:00:00


In [2]:
# Import necessary libraries
import pandas as pd
import numpy as np
import seaborn as sns
from matplotlib import pyplot as plt
import plotly.express as px
import plotly.graph_objects as go
import plotly.figure_factory as ff

# Import machine learning libraries
import lightgbm as lgb
import xgboost as xgb
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import (
    accuracy_score,
    precision_score,
    recall_score,
    f1_score,
    roc_auc_score,
    roc_curve,
    confusion_matrix
)
from sklearn.model_selection import train_test_split, RandomizedSearchCV, cross_val_score
from sklearn.impute import SimpleImputer
from sklearn.feature_selection import mutual_info_classif

In [ ]:
from google.colab import auth
auth.authenticate_user()
print("Authenticated")

In [ ]:
PROJECT_ID = 'prj-snd-ew3-vschmitt'

In [ ]:
from google.cloud import bigquery
client = bigquery.Client(project=PROJECT_ID)

In [ ]:
BQ_PROJECT = PROJECT_ID
BQ_DATASET = 'lead_prediction'
BQ_TABLE = 'trainings_data'

In [ ]:
# SQL query to select data
query = f"""
SELECT *
FROM `{BQ_PROJECT}.{BQ_DATASET}.{BQ_TABLE}`
"""

try:
    # Execute the query and convert the result to a pandas DataFrame
    df = client.query(query=query).to_dataframe()
    print("Query executed successfully and DataFrame is ready.")
except Exception as e:
    print(f"An error occurred: {e}")

In [ ]:
df.shape

In [ ]:
df.head()

In [ ]:
df.drop(['Lead_ID',], axis="columns", inplace=True)

In [ ]:
df.rename(columns={'Lead_Source' : 'Lead Source', 'Lead_Source_Detail' : 'Lead Source Detail',
                   'Name_Case' : 'Name Case', 'has_Phone' : 'has Phone', 'Accept_Data_Policy' : 'Accept Data Policy',
                   'Conversion_Type' : 'Conversion Type', 'Study_Model' : 'Study Model', 'Type_of_Study_Program' : 'Type of Study Program',
                   'Study_Location' : 'Study Location', 'Study_Program' : 'Study Program', 'Sum_of_Study_Interests' : 'Sum of Study Interests' , 'Domain_Type' : 'Domain Type' },inplace=True)


In [ ]:
df.head()

In [ ]:
df.isna().sum().to_frame(name='Missing Values').sort_values('Missing Values', ascending=False)

In [ ]:
# Calculate the percentage of missing values for each column
percentage_missing = df.isnull().sum() * 100 / len(df)
# Create a DataFrame to display the missing values
missing_values_df = percentage_missing.to_frame(name='Missing Values').sort_values('Missing Values', ascending=False)
# Display the DataFrame
missing_values_df

In [ ]:
df['Host'].value_counts()

In [ ]:
unwanted_hosts = ['staging.hs-fresenius.de', 'hsf035.dev.360vier.net', 'b2uxwopd.myraidbox.de', 'digital-health-school.de']

# Filter the DataFrame
df = df[~df['Host'].isin(unwanted_hosts)]

In [ ]:
unwanted_Lead_Source = ['-149307507Publisher', 'Not Found']

# Filter the DataFrame
df = df[~df['Lead Source'].isin(unwanted_Lead_Source )]

In [ ]:
df['Conversion Type'] = df['Conversion Type'].fillna('Infomaterial')

In [ ]:
df['Gender'] = df['Gender'].fillna('weiblich')

In [ ]:
df['Host'] = df['Host'].fillna('www.hs-fresenius.de')

In [ ]:
df.describe()

In [ ]:
df.isna().sum().to_frame(name='Missing Values').sort_values('Missing Values', ascending=False)

In [ ]:
plt.figure(figsize=(5,5))
sns.countplot(x='is_converted', data=df)
plt.title('No of Unsuccessful Leads vs Successful Converted Leads', color='blue')
plt.xticks(np.arange(2), ('Unsuccessful', 'Successful'))
plt.xlabel('Conversion Status')
plt.ylabel('Count')
plt.show()

In [ ]:
total_leads = len(df)
successful_leads = df['is_converted'].sum()
conversion_rate = (successful_leads / total_leads) * 100
print(f'Conversion Rate: {conversion_rate:.2f}%')

In [ ]:
# Creating a crosstab DataFrame for demonstration
crosstab_df = pd.crosstab(index=df['Lead Source'], columns='count')

In [ ]:
def plot_frequency_and_percentage(feature, df, category_df, another_row=False, height=8, ylabels=[]):
    if another_row:
        fig, (ax1, ax2) = plt.subplots(nrows=2, ncols=1, figsize=(10, height * 2))
    else:
        fig, (ax1, ax2) = plt.subplots(ncols=2, figsize=(15, 6))
    fig.subplots_adjust(hspace=0.3, wspace=0.3)

    # Get top 5 categories by frequency
    top_categories = category_df[feature].value_counts().nlargest(5).index

    # Frequency plot for top 5 categories
    ax1.set_title(f'Frequency Plot of {feature}', color='blue')
    ax1.set_ylabel(feature)
    ax1.set_xlabel('count')
    sns.countplot(y=feature, data=category_df[category_df[feature].isin(top_categories)],
                  order=top_categories, ax=ax1, color='green')
    if ylabels:
        ax1.set_yticklabels(ylabels)

    # Calculate count of converted leads for top 5 categories
    converted_counts = df[df['is_converted'] == True][feature].value_counts()
    converted_counts = converted_counts[converted_counts.index.isin(top_categories)]

    # Converted leads plot for top 5 categories
    ax2.set_title('Converted Leads Count', color='blue')
    ax2.set_ylabel(feature)
    ax2.set_xlabel('count')
    converted_counts.sort_values().plot(kind='barh', ax=ax2, color='orange')
    if ylabels:
        ax2.set_yticklabels(ylabels)

    # Calculate and display overall conversion rate
    total_leads = len(df)
    successful_leads = df['is_converted'].sum()
    conversion_rate = (successful_leads / total_leads) * 100
    fig.suptitle(f'Overall Conversion Rate: {conversion_rate:.2f}%', fontsize=14, color='red')


plt.show()

In [ ]:

def plot_conversion_rate(df, is_converted_column, feature):
    """
    Plots the conversion rate for a specified feature.

    Parameters:
    df (DataFrame): The dataframe containing the data.
    is_converted_column (str): The name of the column indicating if a conversion occurred.
    feature (str): The feature for which to calculate and plot the conversion rate.
    """
    # Filter data where is_converted is True
    df_converted = df[df[is_converted_column] == True]

    # Calculate the count of conversions for each unique value in the feature
    converted_counts = df_converted[feature].value_counts()

    # Calculate the percentage of each unique value within the converted data
    conversion_rate = (converted_counts / converted_counts.sum()) * 100
    conversion_rate = conversion_rate.sort_values(ascending=False)

    # Plot bar chart
    plt.figure(figsize=(10, 6))
    conversion_rate.plot(kind='bar')
    plt.title(f'Percentage of Converted Users by {feature}')
    plt.xlabel(feature)
    plt.ylabel('Percentage of Converted Users (%)')
    plt.xticks(rotation=45)
    plt.tight_layout()
    plt.show()

In [ ]:
plot_frequency_and_percentage('Lead Source', df, df, True, 6)

In [ ]:
plot_conversion_rate(df, 'is_converted', 'Lead Source')

In [ ]:
#plot_frequency_and_percentage('Lead_Source_Detail', df, df, True, 6)
plot_frequency_and_percentage('Lead Source Detail', df, df, another_row=False, height=8, ylabels=[])

In [ ]:
plot_frequency_and_percentage('Gender', df, df, True, 6)

In [ ]:
plot_frequency_and_percentage('Study Program', df, df, True, 6)

In [ ]:
plot_frequency_and_percentage('Study Model', df, df, True, 6)

In [ ]:
plot_conversion_rate(df, 'is_converted', 'Study Model')

In [ ]:
plot_frequency_and_percentage('Accept Data Policy', df, df, True, 6)

In [ ]:
plot_conversion_rate(df, 'is_converted', 'Accept Data Policy')

In [ ]:
plot_conversion_rate(df, 'is_converted', 'Name Case')

In [ ]:
plot_frequency_and_percentage('Name Case', df, df, True, 6)

In [ ]:
plot_conversion_rate(df, 'is_converted', 'Domain Type')

In [ ]:
filtered_df = df[df['is_converted'] == True]

# Step 3: Calculate the mean of 's_onTime' for the filtered DataFrame
s_onTime_mean_filtered = filtered_df['Sum of Study Interests'].mean()
print(s_onTime_mean_filtered)

In [ ]:
plot_frequency_and_percentage('has Phone', df, df, True, 6)

In [ ]:
plot_frequency_and_percentage('Name Case', df, df, True, 6)

In [ ]:
plot_frequency_and_percentage('Conversion Type', df, df, True, 6)

In [ ]:
plot_conversion_rate(df, 'is_converted', 'Conversion Type')

In [ ]:
# Import necessary libraries
import lightgbm as lgb
import xgboost as xgb
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, roc_curve, confusion_matrix
from sklearn.model_selection import train_test_split, RandomizedSearchCV, cross_val_score
from imblearn.over_sampling import SMOTENC
from sklearn.preprocessing import LabelEncoder, StandardScaler, OrdinalEncoder
import category_encoders as ce

In [ ]:
df.dtypes

In [ ]:
df.nunique()

In [ ]:
# Create a copy of the dataframe to work with
df_encoded = df.copy()

# Step 1: Drop 'is_converted' column from df_encoded and assign to X_
X = df_encoded.drop(columns=['is_converted'])

# Step 2: Extract 'is_converted' column from df_encoded and assign to y
y = df_encoded['is_converted']

In [ ]:
# Step 3: Label Encoding for specified column

#binary_features = ['Lead Source Detail', 'Study Program']
##encoder_target = ce.TargetEncoder(cols=binary_features)
#X = encoder_target.fit_transform(X, y)

one_hot_features = ['Host', 'Conversion Type', 'Gender', 'Study Model',  'Faculty', 'Study Location', 'Type of Study Program',
                  'Lead Source', 'Accept Data Policy', 'has Phone', 'Domain Type', 'Name Case', 'Lead Source Detail', 'Study Program', 'Semester']

X = pd.get_dummies(X, columns=one_hot_features)



# Step 6: Scaling the dataframe
scaler = StandardScaler()
df_scaled = pd.DataFrame(scaler.fit_transform(X), columns=X.columns)

# Step 7: Define the encoded and scaled dataframe as X_prep
X_en = df_scaled

In [ ]:
# Step 4: Identify categorical features indices for SMOTENC
#categorical_columns = [col for col in X.columns if col not in ['Sum of Study Interests']]

#all_columns = X_en.columns.tolist()
#categorical_indices = [all_columns.index(col) for col in categorical_columns if col in all_columns]


In [ ]:
lb = LabelEncoder()
lb.fit(y)
y_encoded = lb.transform(y)
print("Encoded labels:",lb.classes_)
y_en = pd.Series(y_encoded)

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_en, y_en, test_size=0.2, stratify=y, random_state=42)


print("Training Data Shape : ", X_train.shape, y_train.shape)
print("Test Data Shape : ", X_test.shape, y_test.shape)

In [ ]:
from imblearn.over_sampling import SMOTE
from sklearn.neighbors import NearestNeighbors
import pandas as pd
import time
'''
# Check available columns in X_train
print("Columns in X_train:", X_train.columns.tolist())

# Define the list of original categorical columns before one-hot encoding
original_categorical_columns = [
    'Host', 'Conversion Type', 'Gender', 'Study Model', 'Faculty', 'Study Location', 'Type of Study Program',
    'Lead Source', 'Accept Data Policy', 'has Phone', 'Domain Type', 'Name Case', 'Lead Source Detail',
    'Study Program', 'Semester'
]

# Function to get one-hot encoded columns for a given original categorical column
def get_one_hot_encoded_columns(original_column_name, all_columns):
    return [col for col in all_columns if col.startswith(original_column_name + '_')]

# Extract indices of one-hot encoded columns
categorical_features_indices = []
missing_columns = []

for original_col in original_categorical_columns:
    encoded_columns = get_one_hot_encoded_columns(original_col, X_train.columns)
    if encoded_columns:
        categorical_features_indices.extend([X_train.columns.get_loc(col) for col in encoded_columns])
    else:
        missing_columns.append(original_col)

if missing_columns:
    print("Warning: The following original columns have no corresponding one-hot encoded columns in X_train:", missing_columns)

# Proceed if there are valid categorical features

'''
'''
categorical_columns = [col for col in X.columns if col not in ['Sum of Study Interests']]

all_columns = X_en.columns.tolist()
categorical_indices = [all_columns.index(col) for col in categorical_columns if col in all_columns]

if categorical_indices:
   # Nearest Neighbors Estimator mit Parallelisierung
    nn_estimator = NearestNeighbors(n_neighbors=5, n_jobs=-1)

    # Initialize SMOTENC with Nearest Neighbors Estimator
    smotenc = SMOTENC(categorical_features=categorical_indices, random_state=42, k_neighbors=nn_estimator)

    # Start timing
    print("Start der Berechnung von SMOTENC...")
    start_time = time.time()

    # Fit and resample the training data
    X_train, y_train = smotenc.fit_resample(X_train, y_train)

    # End timing
    end_time = time.time()
    print("SMOTENC Berechnung abgeschlossen.")
    print(f"Dauer der Berechnung: {end_time - start_time:.2f} Sekunden")
else:
    print("No valid categorical features found. SMOTENC will not be applied.")
    '''
# Start timing
print("Start der Berechnung von SMOTE...")
start_time = time.time()

# Initialize SMOTE
smote = SMOTE(random_state=42, k_neighbors=5, n_jobs=-1)

# Fit and resample the training data
X_train, y_train = smote.fit_resample(X_train, y_train)

# End timing
end_time = time.time()
print("SMOTE Berechnung abgeschlossen.")
print(f"Dauer der Berechnung: {end_time - start_time:.2f} Sekunden")


In [ ]:
print("Smote Train Data : ", X_train.shape, y_train.shape)

In [ ]:
# Compute Mutual Information scores
mi_classification = mutual_info_classif(X_train, y_train, discrete_features='auto')

# Create a pandas Series with the MI scores and the feature names
mi_series = pd.Series(mi_classification, index=X_train.columns)

# Exclude the specified column
mi_series = mi_series.drop(labels='Sum of Study Interests')

# Sort the MI scores in descending order and convert to a DataFrame
data_mi = mi_series.sort_values(ascending=False).to_frame(name='MI')

# Display the DataFrame
print(data_mi)


In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# Function to plot the Mutual Information scores
def plot_mi_scores(scores):
    # Select top 10 scores
    top_scores = scores.sort_values(ascending=False).head(30)
    sorted_scores = top_scores.sort_values(ascending=True)
    width = np.arange(len(sorted_scores))
    ticks = sorted_scores.index
    plt.barh(width, sorted_scores)
    plt.yticks(width, ticks)
    plt.title("Top 10 Mutual Information Scores")
    plt.xlabel("Score")
    plt.ylabel("Features")

    # Add text annotations for the bar values
    for index, value in enumerate(sorted_scores):
        plt.text(value, index, f'{value:.2f}', va='center', ha='left')

    plt.show()

# Plot the Mutual Information scores
plt.figure(dpi=100, figsize=(8, 5))
plot_mi_scores(mi_series)



In [ ]:
one_hot_features = ['Host', 'Conversion Type', 'Gender', 'Study Model',  'Faculty', 'Study Location', 'Type of Study Program',
                  'Lead Source', 'Accept Data Policy', 'has Phone', 'Domain Type', 'Name Case', 'Lead Source Detail', 'Study Program', 'Semester']  # Replace with actual prefixes
aggregated_mi = {}
for col in mi_series.index:
    if any(prefix in col for prefix in one_hot_features):
        original_col = next(prefix for prefix in one_hot_features if prefix in col)
        if original_col not in aggregated_mi:
            aggregated_mi[original_col] = 0
        aggregated_mi[original_col] += mi_series[col]
    else:
        aggregated_mi[col] = mi_series[col]

# Convert the aggregated MI scores to a DataFrame
aggregated_mi_series = pd.Series(aggregated_mi)
data_mi = aggregated_mi_series.sort_values(ascending=False).to_frame(name='MI')

# Display the DataFrame
print(data_mi)

In [ ]:
plt.figure(figsize=(10, 6))
data_mi['MI'].plot(kind='bar')
plt.title('Feature Importance using Mutual Information')
plt.xlabel('Features')
plt.ylabel('Mutual Information Score')
plt.show()

In [ ]:
import pandas as pd
import statsmodels.api as sm
from statsmodels.formula.api import ols
from scipy import stats

# Assuming 'Sum of Study Interests' is a column in X_train
# Extract the specific feature
feature_name = 'Sum of Study Interests'
X_feature = X_train[feature_name]

# Assuming 'is_converted' is a column in y_train
target_name = 'is_converted'
y_target = y_train[target_name]

# Combine into a DataFrame
data = pd.DataFrame({
    'Feature': X_feature,
    'Target': y_target
})

# Perform ANOVA using statsmodels
model = ols('Target ~ C(Feature)', data=data).fit()
anova_table = sm.stats.anova_lm(model, typ=2)

print("Detailed ANOVA using statsmodels")
print(anova_table)

# Perf


In [ ]:
from sklearn.model_selection import cross_validate, StratifiedKFold
cv_split = StratifiedKFold(n_splits=10)

In [ ]:
def evaluate_model(model, X_test, y_test, model_name):
    predictions = model.predict(X_test)
    proba = model.predict_proba(X_test)[:, 1]

    accuracy = accuracy_score(y_test, predictions)
    precision = precision_score(y_test, predictions)
    recall = recall_score(y_test, predictions)
    f1 = f1_score(y_test, predictions)
    roc_auc = roc_auc_score(y_test, proba)

    # Confusion Matrix
    cm = confusion_matrix(y_test, predictions)
    tn, fp, fn, tp = cm.ravel()
    specificity = tn / (tn + fp)

    # Cross Validation
    cv_scores = cross_val_score(model, X_train, y_train, cv=5, scoring='accuracy')

    print(f"{model_name} Metrics:")
    print(f"Accuracy: {accuracy}")
    print(f"Precision: {precision}")
    print(f"Recall (Sensitivity): {recall}")
    print(f"F1-Score: {f1}")
    print(f"Specificity: {specificity}")
    print(f"ROC AUC: {roc_auc}")
    print(f"Cross Validation Accuracy: {cv_scores.mean()}")

    # ROC Curve
    fpr, tpr, _ = roc_curve(y_test, proba)
    plt.figure(figsize=(10, 6))
    plt.plot(fpr, tpr, color='blue', label=f"{model_name} (AUC = {roc_auc:.2f})")
    plt.plot([0, 1], [0, 1], color='red', linestyle='--')
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title(f'{model_name} ROC Curve')
    plt.legend()
    plt.show()

    # Confusion Matrix Visualization
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=['Not Converted', 'Converted'], yticklabels=['Not Converted', 'Converted'])
    plt.xlabel('Predicted')
    plt.ylabel('Actual')
    plt.title(f'{model_name} Confusion Matrix')
    plt.show()

    return {
        'accuracy': accuracy,
        'precision': precision,
        'recall': recall,
        'f1': f1,
        'roc_auc': roc_auc,
        'specificity': specificity,
        'cv_accuracy': cv_scores.mean()
    }


In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
import xgboost as xgb
# Pipelines for each model without hyperparameter tuning

pipeline_xgb = Pipeline([
    ('clf', xgb.XGBClassifier())
])

pipeline_rf = Pipeline([
    ('clf', RandomForestClassifier())
])


pipeline_dt = Pipeline([
    ('clf', DecisionTreeClassifier())
])



In [ ]:

model_performance = {}

In [ ]:
# Train and evaluate XGBoost
model_performance['XGBoost'] = evaluate_model(pipeline_xgb.fit(X_train, y_train),X_test, y_test, "XGBoost")


In [ ]:
# Train and evaluate RandomForest
model_performance['RandomForest'] = evaluate_model(pipeline_rf.fit(X_train, y_train), X_test, y_test, "RandomForest")

In [ ]:
# Train and evaluate DecisionTree
model_performance['DecisionTree'] = evaluate_model(pipeline_dt.fit(X_train, y_train), X_test, y_test, "DecisionTree")


In [ ]:
# Convert the model performance dictionary to a DataFrame
results_df = pd.DataFrame(model_performance).T

# Save the DataFrame to an Excel file
results_df.to_excel('model_performance_results5.xlsx', index=True)
print("Model performance results saved to 'model_performance_results_final.xlsx'.")

In [ ]:
# Determine the best model based on ROC AUC
best_model_name = max(model_performance, key=lambda k: model_performance[k]['roc_auc'])
print(f"The best model is {best_model_name} based on ROC AUC.")


In [ ]:
 #Define hyperparameters for Grid Search


param_grid_xgb = {
    #'clf__max_depth': [3, 5, 7],
    #'clf__learning_rate': [0.1, 0.01],
    #'clf__n_estimators': [100, 200]

    'clf__max_depth': [3, 5, 7, 9, 11],
    'clf__learning_rate': [0.1, 0.01, 0.001],
    'clf__n_estimators': [100, 200, 300],
    'clf__subsample': [0.6, 0.8, 1.0],
    'clf__colsample_bytree': [0.6, 0.8, 1.0],
    'clf__gamma': [0, 0.1, 0.2],
    'clf__min_child_weight': [1, 3, 5],
    'clf__reg_alpha': [0, 0.1, 0.5],
    'clf__reg_lambda': [1, 1.5, 2]
}


param_grid_rf = {
    #'clf__n_estimators': [100, 200],
    #'clf__max_depth': [None, 10, 20]
    'clf__n_estimators': [100, 200, 300, 500],
    'clf__max_depth': [None, 10, 20, 30],
    'clf__min_samples_split': [2, 5, 10],
    'clf__min_samples_leaf': [1, 2, 4],
    'clf__max_features': ['auto', 'sqrt', 'log2'],
    'clf__bootstrap': [True, False]
}

param_grid_dt = {
    #'clf__max_depth': [None, 10, 20],
    #'clf__min_samples_split': [2, 10, 20]
    'clf__max_depth': [None, 10, 20, 30],
    'clf__min_samples_split': [2, 10, 20, 30],
    'clf__min_samples_leaf': [1, 2, 4, 6],
    'clf__max_features': ['auto', 'sqrt', 'log2']
}

In [ ]:
from sklearn.model_selection import GridSearchCV
#Function to perform Grid Search
def perform_grid_search(pipeline, param_grid, X_train, y_train):
    grid_search = GridSearchCV(estimator=pipeline, param_grid=param_grid, cv=5, scoring='roc_auc', n_jobs=-1, verbose=2)
    grid_search.fit(X_train, y_train)
    return grid_search

In [ ]:
# Perform Grid Search for each model
#grid_search_lgb = perform_grid_search(pipeline_lgb, param_grid_lgb, X_train, y_train)
grid_search_xgb = perform_grid_search(pipeline_xgb, param_grid_xgb, X_train, y_train)
grid_search_rf = perform_grid_search(pipeline_rf, param_grid_rf, X_train, y_train)
grid_search_dt = perform_grid_search(pipeline_dt, param_grid_dt, X_train, y_train)


In [ ]:
# Evaluate models and store performance
model_performance = {}
# Function to evaluate model
def evaluate_model(grid_search, X_test, y_test, model_name, X_train, y_train):
    best_model = grid_search.best_estimator_
    y_pred = best_model.predict(X_test)
    y_pred_proba = best_model.predict_proba(X_test)[:, 1]

    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred, average='weighted')
    recall = recall_score(y_test, y_pred, average='weighted')
    f1 = f1_score(y_test, y_pred, average='weighted')
    roc_auc = roc_auc_score(y_test, y_pred_proba, multi_class='ovo')

    cm = confusion_matrix(y_test, y_pred)
    tn = cm[0][0]
    fp = cm[0][1] if len(cm[0]) > 1 else 0
    specificity = tn / (tn + fp) if (tn + fp) > 0 else 0

    cv_scores = cross_val_score(best_model, X_train, y_train, cv=5, scoring='accuracy')

    print(f"{model_name} Metrics:")
    print(f"Accuracy: {accuracy}")
    print(f"Precision: {precision}")
    print(f"Recall (Sensitivity): {recall}")
    print(f"F1-Score: {f1}")
    print(f"Specificity: {specificity}")
    print(f"ROC AUC: {roc_auc}")
    print(f"Cross Validation Accuracy: {cv_scores.mean()}")

    # ROC Curve
    fpr, tpr, _ = roc_curve(y_test, y_pred_proba)
    plt.figure(figsize=(10, 6))
    plt.plot(fpr, tpr, color='blue', label=f"{model_name} (AUC = {roc_auc:.2f})")
    plt.plot([0, 1], [0, 1], color='red', linestyle='--')
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title(f'{model_name} ROC Curve')
    plt.legend()
    plt.show()

    # Confusion Matrix Visualization
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=['Class 0', 'Class 1'], yticklabels=['Class 0', 'Class 1'])
    plt.xlabel('Predicted')
    plt.ylabel('Actual')
    plt.title(f'{model_name} Confusion Matrix')
    plt.show()

    return {
        'accuracy': accuracy,
        'precision': precision,
        'recall': recall,
        'f1': f1,
        'roc_auc': roc_auc,
        'specificity': specificity,
        'cv_accuracy': cv_scores.mean(),
        'best_params': grid_search.best_params_
    }

In [ ]:
model_performance['XGBoost'] = evaluate_model(grid_search_xgb, X_test, y_test, 'XGBoost', X_train, y_train)

In [ ]:
model_performance['RandomForest'] = evaluate_model(grid_search_rf, X_test, y_test, 'RandomForest', X_train, y_train)

In [ ]:
model_performance['DecisionTree'] = evaluate_model(grid_search_dt, X_test, y_test, 'DecisionTree', X_train, y_train)

In [ ]:
# Convert the model performance dictionary to a DataFrame
results_df = pd.DataFrame(model_performance).T

# Save results to an Excel file
results_df.to_excel('model_performance_metrics_final.xlsx', index=True)

# Print results for each model
print("Model Performance Metrics:")
print(results_df)


In [ ]:
query = """
SELECT *
FROM `prj-snd-ew3-vschmitt.lead_prediction.validation_data`
"""

try:
    df = client.query(query=query).to_dataframe()
    print("Query executed successfully and DataFrame is ready.")
except Exception as e:
    print(f"An error occurred: {e}")

In [ ]:
target_column = 'is_converted'
X_new = df.drop(columns=[target_column])
y_new = df[target_column]
scaler = StandardScaler()
X_new_scaled = scaler.fit_transform(X_new)

In [ ]:
import pickle
model_path = 'path_to_your_saved_model.pkl'
with open(model_path, 'rb') as file:
    model = pickle.load(file)

In [ ]:
predictions = model.predict(X_new_scaled)
results_df = pd.DataFrame({
    'Predictions': predictions,
    #'True Labels': y_new  # Optional: include true labels if available
})


In [ ]:
destination_table = 'prj-snd-ew3-vschmitt.lead_prediction.validation_data_model'
try:
    results_df.to_gbq(destination_table, if_exists='replace')
    print(f"Results successfully saved to {destination_table}.")
except Exception as e:
    print(f"An error occurred while saving results to BigQuery: {e}")